> Projeto Desenvolve <br>
Programação Intermediária com Python <br>
Profa. Camila Laranjeira (mila@projetodesenvolve.com.br) <br>

# 3.14 - ORM

## Exercícios

#### Q1. Conhecendo os dados
Baixe o seguinte csv onde iremos trabalhar. Ele contém informações sobre salários de profissionais de dados de uma empresa hipotética entre 2009 e 2016
* https://github.com/camilalaranjeira/python-intermediario/blob/main/salaries.csv

Suas colunas, descritas na [página do Kaggle que contém o dataset](https://www.kaggle.com/datasets/krishujeniya/salary-prediction-of-data-professions?resource=download), são:
* FIRST NAME: Primeiro nome do profissional de dados (String)
* LAST NAME: Sobrenome do profissional de dados (String)
* SEX: Gênero do profissional de dados (String: 'F' para Feminino, 'M' para Masculino)
* DOJ (Date of Joining): A data em que o profissional de dados ingressou na empresa (Data no formato MM/DD/AAAA)
* CURRENT DATE: A data atual ou a data de referência dos dados (Data no formato MM/DD/AAAA)
* DESIGNATION: O cargo ou designação do profissional de dados (String: ex., Analista, Analista Sênior, Gerente)
* AGE: Idade do profissional de dados (Integer)
* SALARY: Salário anual do profissional de dados (Float)
* UNIT: Unidade de negócios ou departamento em que o profissional de dados trabalha (String: ex., TI, Finanças, Marketing)
* LEAVES USED: Número de licenças utilizadas pelo profissional de dados (Integer)
* LEAVES REMAINING: Número de licenças restantes para o profissional de dados (Integer)
* RATINGS: Avaliações de desempenho do profissional de dados (Float)
* PAST EXP: Experiência de trabalho anterior em anos antes de ingressar na empresa atual (Float)

Na célula a seguir, **carregue os dados do CSV e dê uma olhada neles antes de seguir**.

In [1]:
### Escreva sua resposta aqui
import pandas as pd
df = pd.read_csv('salaries.csv')

#### Q2. Modelando os dados
Você deve **criar um ORM com SQLAlchemy capaz de comportar os dados dessa base**.

* Crie um campo de chave primária `ID`, que deve ser incrementado automaticamente
* Os campos SEX, DESIGNATION e UNIT devem ser definidos como classes `Enum` com os possíveis valores (consulte os valores únicos dessas colunas)
* Para os outros campos, consulte os tipos de dados informados na descrição acima

In [2]:
### Escreva sua resposta aqui
from sqlalchemy import Column, Integer, String, Float, Enum
from sqlalchemy.ext.declarative import declarative_base
import enum

Base = declarative_base()

# Exemplo de Enum para SEX
class SexEnum(enum.Enum):
    MALE = "Male"
    FEMALE = "Female"

# Exemplo de Enum para DESIGNATION
class DesignationEnum(enum.Enum):
    INTERN = "Intern"
    MANAGER = "Manager"
    DIRECTOR = "Director"

# Exemplo de Enum para UNIT
class UnitEnum(enum.Enum):
    HR = "HR"
    SALES = "Sales"
    TECH = "Tech"

# Classe ORM
class Employee(Base):
    __tablename__ = 'employees'

    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String)
    age = Column(Integer)
    salary = Column(Float)
    sex = Column(Enum(SexEnum))
    designation = Column(Enum(DesignationEnum))
    unit = Column(Enum(UnitEnum))



C:\Users\PDITA157\AppData\Local\Temp\ipykernel_17560\605145113.py:6: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


#### Q3. Estabelecendo uma conexão

Usando o método `create_engine` do SQLAlchemy, crie uma conexão com um novo banco de dados SQLite chamado `salarios`.

In [3]:
### Escreva sua resposta aqui
from sqlalchemy import create_engine, URL
from sqlalchemy import text

engine = create_engine("sqlite+pysqlite:///salarios.sqlite", echo=True)

#### Q4. Criando as tabelas
Crie as tabelas da questão Q2 no banco `salarios`.

In [4]:
### Escreva sua resposta aqui
# Cria as tabelas no banco
Base.metadata.create_all(engine)

2025-05-14 20:33:06,608 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-14 20:33:06,610 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("employees")
2025-05-14 20:33:06,614 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-14 20:33:06,616 INFO sqlalchemy.engine.Engine COMMIT


#### Q5. Populando

Usando o método `to_sql` da biblioteca Pandas (veja [a documentação](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html)), popule o banco `salarios` com os dados do csv que você carregou na questão Q1.
* Lembre-se de definir o parâmetro `if_exists='append'` para que as tabelas não sejam dropadas e recriadas.

In [10]:
### Escreva sua resposta aqui
df.to_sql(name="salaries", con=engine, if_exists="append", index=False)

print("Dados inseridos com sucesso!")

2025-05-14 20:35:07,534 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-14 20:35:07,544 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("salaries")
2025-05-14 20:35:07,544 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-14 20:35:07,544 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("salaries")
2025-05-14 20:35:07,544 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-14 20:35:07,554 INFO sqlalchemy.engine.Engine 
CREATE TABLE salaries (
	"FIRST NAME" TEXT, 
	"LAST NAME" TEXT, 
	"SEX" TEXT, 
	"DOJ" TEXT, 
	"CURRENT DATE" TEXT, 
	"DESIGNATION" TEXT, 
	"AGE" FLOAT, 
	"SALARY" BIGINT, 
	"UNIT" TEXT, 
	"LEAVES USED" FLOAT, 
	"LEAVES REMAINING" FLOAT, 
	"RATINGS" FLOAT, 
	"PAST EXP" BIGINT
)


2025-05-14 20:35:07,559 INFO sqlalchemy.engine.Engine [no key 0.00280s] ()
2025-05-14 20:35:07,624 INFO sqlalchemy.engine.Engine INSERT INTO salaries ("FIRST NAME", "LAST NAME", "SEX", "DOJ", "CURRENT DATE", "DESIGNATION", "AGE", "SALARY", "UNIT", "LEAVES USED", "LEAVES REMAINING",

#### Q6. Consultas SQL vs ORM

Agrupe os dados por DESIGNATION e selecione o mínimo, máximo e a média dos salários (SALARY) divididos por 12. Já que o atributo SALARY é anual, dividir por 12 nos mostrará os valores mensais.

Assumindo que a variável que armazena a sua conexão se chama `engine`, você deve realizar a query acima de três formas:
* Executando a query SQL através de uma instância de conexão retornada pelo método `engine.connect()`
* Executando a query SQL com o método `read_sql_query` do Pandas (veja [a documentação](https://pandas.pydata.org/docs/reference/api/pandas.read_sql_query.html)). Você usará mesma instância `engine.connect()` como um dos parâmetros.
* Executando uma query criada com o módulo `select` do SQLAlchemy. Sua execução deve ser feita através de um objeto `Session` do módulo `orm` do SQLAlchemy (`Session(engine)`).


In [11]:
### Execute aqui sua query SQL com SQLAlchemy
from sqlalchemy import text

# Cria a conexão
with engine.connect() as conn:
    result = conn.execute(text("""
        SELECT 
            designation,
            MIN(salary / 12.0) AS min_sal_mensal,
            MAX(salary / 12.0) AS max_sal_mensal,
            AVG(salary / 12.0) AS media_sal_mensal
        FROM employees
        GROUP BY designation
    """))

    for row in result:
        print(row)


2025-05-14 20:36:07,634 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-14 20:36:07,637 INFO sqlalchemy.engine.Engine 
        SELECT 
            designation,
            MIN(salary / 12.0) AS min_sal_mensal,
            MAX(salary / 12.0) AS max_sal_mensal,
            AVG(salary / 12.0) AS media_sal_mensal
        FROM employees
        GROUP BY designation
    
2025-05-14 20:36:07,639 INFO sqlalchemy.engine.Engine [generated in 0.00597s] ()
2025-05-14 20:36:07,644 INFO sqlalchemy.engine.Engine ROLLBACK


In [12]:
### Execute aqui sua query SQL com SQLAlchemy + Pandas
import pandas as pd

with engine.connect() as conn:
    df = pd.read_sql_query("""
        SELECT 
            designation,
            MIN(salary / 12.0) AS min_sal_mensal,
            MAX(salary / 12.0) AS max_sal_mensal,
            AVG(salary / 12.0) AS media_sal_mensal
        FROM employees
        GROUP BY designation
    """, conn)

print(df)


2025-05-14 20:38:16,674 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-14 20:38:16,674 INFO sqlalchemy.engine.Engine 
        SELECT 
            designation,
            MIN(salary / 12.0) AS min_sal_mensal,
            MAX(salary / 12.0) AS max_sal_mensal,
            AVG(salary / 12.0) AS media_sal_mensal
        FROM employees
        GROUP BY designation
    
2025-05-14 20:38:16,676 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-14 20:38:16,696 INFO sqlalchemy.engine.Engine ROLLBACK
Empty DataFrame
Columns: [designation, min_sal_mensal, max_sal_mensal, media_sal_mensal]
Index: []


In [13]:
### Execute aqui sua query com SQLAlchemy ORM
from sqlalchemy import select, func
from sqlalchemy.orm import Session

with Session(engine) as session:
    stmt = (
        select(
            Employee.designation,
            func.min(Employee.salary / 12).label("min_sal_mensal"),
            func.max(Employee.salary / 12).label("max_sal_mensal"),
            func.avg(Employee.salary / 12).label("media_sal_mensal"),
        )
        .group_by(Employee.designation)
    )

    results = session.execute(stmt)

    for row in results:
        print(row)


2025-05-14 20:40:06,191 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-14 20:40:06,200 INFO sqlalchemy.engine.Engine SELECT employees.designation, min(employees.salary / (? + 0.0)) AS min_sal_mensal, max(employees.salary / (? + 0.0)) AS max_sal_mensal, avg(employees.salary / (? + 0.0)) AS media_sal_mensal 
FROM employees GROUP BY employees.designation
2025-05-14 20:40:06,202 INFO sqlalchemy.engine.Engine [generated in 0.00163s] (12, 12, 12)
2025-05-14 20:40:06,206 INFO sqlalchemy.engine.Engine ROLLBACK
